<img src="./header.png" width=55%>

<img src="https://cdnp3.stackassets.com/42e3d5100c4c83162def7a690cf51acda1631b64/store/opt/596/298/58be047314c354bc0b2bd925b58e873c4d0b0f9a7ebc5ea2eff2ec3d6242/27dec723e2238e11de6db0dfa8ad9daaf62bac50_main_hero_image.jpg" width=55%>


<h1 align="center">Análisis de Datos y Visualizaciones con Python</h1>
<center>
        <b>Presentadores:</b>
<br>
<br> 
<i><b>Sebastián Bórquez</b> - <a href="mailto:sebastian.borquez.g@gmail.com">sebastian.borquez.g@gmail.com</a> - DI UTFSM</i><br>
    <i><b>Patricio Campaña</b> - <a href="mailto:patricio.campana@sansano.usm.cl">patricio.campana@sansano.usm.cl</a> - DI UTFSM</i><br>
</center>


# Temario

* [0.- ¿Por qué hacer análisis exploratorio?](#intro)
* [1.- El ambiente de desarrollo](#tools)
* [2.- Caso 1: Migración en Chile](#caso1)
* [3.- Caso 2: Yahoo Finances](#caso2)
* [4.- Caso 3: Imágenes por Resonancia Magnética](#caso3)
* [5.- Resumen](#summary)
* [6.- Recursos en Internet](#sources)

<div id="intro"></div>
    
# ¿Por qué hacer análisis exploratorio?

<img src="./larutadeladata.jpg" width=30%></img>


Análisis exploratorio y visualizacion de datos son dos pilares fundamentales del toobox de un data scientist. 

En esta charla utilizaremos Python para aprenderemos como a través de la exploraración y la visualización podemos descubrir que historia cuentan los datos.


<div id="tools"></div>
    
# El ambiente de desarrollo

## Jupyter notebooks

<img width="60%" src="https://cdn-images-1.medium.com/max/1600/1*LPnY8nOLg4S6_TG0DEXwsg.png"></img>

Jupyter Notebook es una herramienta extremadamente eficiente, ya que permite unir __código__ y __texto__. Así cada funcionalidad se puede explicar en detalle. También puede generar gráficos "vivos" generados en tiempo real dentro de la herramienta además de tener soporte para __Markdown__ y __HTML__.

Esto nos permite hacer cosas mágicas como:

In [ ]:
from IPython.display import display, HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/F_VjVqe3KJ0" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

In [ ]:
HTML('''
<div style="margin: 0 auto; width:100%; height:400px;">
    <object type="text/html" data="https://siga.usm.cl/pag/"
            style="width:100%; height:100%; margin:1%;">
    </object>
</div>
''')

## Pandas

<img width="70%" src="https://pandas.pydata.org/_static/pandas_logo.png"></img>

[Pandas](http://pandas.pydata.org/) (_panel data_) es un módulo de Python que nos provee de estructuras de datos, alto desempeño y herramientas para el _análisis de datos_.

Por lo general, como _data scientist_ trabajaremos con datos en forma de tablas, estos por lo general se encuentran almacenados en formatos _csv, xlsx, tsv_, entre otros. Pandas nos ofrece una manera conveniente para _cargar, procesar, analizar y guardar_ dichas tablas.

Las dos estructuras principales que utiliza pandas son las __Series__ y los __Dataframes__.

In [ ]:
import pandas as pd
import numpy as np

Podemos crear series de diferentes tipos

In [ ]:
pd.Series?

In [ ]:
A = pd.Series([1,3,5,2,6,8])
B = pd.Series(np.random.rand(6))
C = pd.Series(pd.Categorical(["x", "x", "y","x","y", "y"]))
D = pd.Series(pd.date_range('2018-01-01', '2018-01-06'))

display(A)
display(B)
display(C)
display(D)

Igualmente, podemos crear Dataframes de diferentes formas

In [ ]:
pd.DataFrame?

In [ ]:
df = pd.DataFrame(np.random.randn(6,4), columns=list('ABCD'))
df

In [ ]:
df = pd.DataFrame({
    "A":A,
    "B":B,
    "C":C,
    "D":D
})
df

Pandas nos permite acceder a los valores de manera flexible, ajustandose a lo que necesitamos

Una columna

In [ ]:
print(type(df["A"]))
df["A"]

Una fila

In [ ]:
print(type(df.loc[3]))
df.loc[3]

In [ ]:
df.iloc[0]

Varias columnas

In [ ]:
print(type(df[["A", "B"]]))
df[["A", "B"]]

Acceder a un valor especifico

In [ ]:
print(type(df.loc[2,"A"]))
df.at[2,"A"]

Varias filas y columnas

In [ ]:
print(type(df.loc[2:5, ["A", "C"]]))
df.loc[2:5, ["A", "C"]]

Podemos cambiar el indice con el que accedemos a las filas

In [ ]:
df = df.set_index("D")
df

In [ ]:
df.loc["2018-01-03"]

Podemos tambien modificar valores

In [ ]:
df.at["2018-01-03", "C"] = "x"
df

O acceder a valores dada una condición

In [ ]:
df[df["C"] == "x"]

Esto continua, pueden acceder a [10 Minutes to Pandas](https://pandas.pydata.org/pandas-docs/stable/10min.html) para mayor explicación de las diferentes operaciones disponibles en Pandas

## Matplotlib

<img width="50%"src="https://matplotlib.org/_static/logo2.png"></img>

Buscaremos ver el comportamiento de la siguiente estimación del interés de los asistentes a esta charla a lo largo del tiempo:

In [ ]:
t = np.arange(0, 60, 0.1)                   # dominio (eje x)
i = np.exp(t - 60) + np.power(-t/60 + 1, 3) # recorrido (eje y)

Intentemos ver su comportamiento ¿Cuáles son los primeros 10 valores de cada arreglo?:

In [ ]:
t[0:10]

In [ ]:
i[0:10]

Estos números no dicen mucho... ¡**[matplotlib](https://matplotlib.org/api/pyplot_api.html)** podría ayudarnos a verlos de mejor manera!

In [ ]:
import matplotlib.pyplot as plt

Podemos graficar con el método `plot` el recorrido `i` sobre el dominio `t`:

In [ ]:
plt.plot(t, i);

Pero aun falta algo... un título y el significado de los ejes:

In [ ]:
plt.title("Interés vs. Tiempo")

plt.xlabel("Tiempo [min]")
plt.ylabel("Interés [pts]")

plt.plot(t, i);

¡Mucho mejor! ¿qué información podemos inferir de este gráfico?

1. El café del break anterior deja de hacer efecto continuamente durante la charla.
2. El siguiente coffe break comienza al rededor de los 55 minutos tras comenzar la charla.

Pero esto es sólo el comienzo...

<div id="caso1"></div>
    
# Caso 1: Migración en Chile
<img src="https://www.extranjeria.gob.cl/media/2016/03/logo-migraciones-sin-fondo-1.png" width=40% height=35%>

El segundo caso consiste de datos de inmigración en Chile durante el año 2018. Estos se encuentran disponibles en la página del __[Departamento de Extranjería y Migración](https://www.extranjeria.gob.cl/estadisticas-migratorias/)__ del Ministerio del Interior y Seguridad Pública. 

_Puedes descargar estos datasets desde este [link](https://www.extranjeria.gob.cl/estadisticas-migratorias/)._

In [ ]:
visas_file = "./data/CH_2018_P.xlsx"

## Cargar datos

In [ ]:
pd.read_excel?

__pd.read_excel()__ nos entrega un __DataFrame__, el cual es la representación que utiliza pandas para una tabla, esta posee tanto los nombres de las columnas, indices y los valores de los datos contenidos en la tabla.

In [ ]:
visas = pd.read_excel(visas_file)

print(type(visas))

## Conociendo el Dataframe

Podemos hacernos una idea del contenido del _Dataframe_ utilizando los métodos __head(n)__ y __tail(n)__ . Tal como sus nombres lo indican, muestran los **n** primeros o __n__ últimas filas del _Dataframe_ respectivamente.

In [ ]:
visas.head(5)

In [ ]:
visas.tail(5)

Vamos a echar un vistazo a la dimensionalidad de los datos, nombres de las columnas y sus tipos.

In [ ]:
print(f"Cantidad de filas: {len(visas)}")
print(f"Dimensiones: {visas.shape}")
print(f"Cantidad de valores: {visas.size}")

In [ ]:
print("Columnas: ", *visas.columns)

In [ ]:
display(visas.info())
display(visas.dtypes)

También podemos revisar cada columna, ahora veremos las diferentes valores que contiene cada columna

In [ ]:
for column in visas:
    print(f"{column} tiene {visas[column].nunique()} valores diferentes")

In [ ]:
print(f"Valores de ESTUDIOS: {visas.ESTUDIOS.unique()}")

In [ ]:
print(f"Cantidad de individuos por cada valor de ESTUDIOS:")
visas.ESTUDIOS.value_counts()

Quizás lo que nos interesa es saber como se distribuyen nuestros individuos pero en dos variables. Para esto existen las _tablas de contigencia_ disponibles con el comando __crosstab__.

In [ ]:
pd.crosstab(visas["ACTIVIDAD"], visas["SEXO"])

## Consultas

Pandas también nos da la opción de realizar consultas, por ejemplo queremos saber solo los datos de los inmigrantes que vienen de _PERÚ_.

In [ ]:
visas[visas["PAIS"] == "PERÚ"].head(5)

O podemos anidar múltiples consultas utilizando __AND, NOT__ y __OR__, para esto usamos __&,|__ y __~__ respectivamente.

In [ ]:
visas[(visas["PAIS"] == "PERÚ") & ((visas["COMUNA"] == "VIÑA DEL MAR") | (visas["COMUNA"] == "VALPARAISO"))].head(5)

Tambien podemos utilizar __isin__ para corroborar con una lista de valores.

In [ ]:
visas[(visas["ACTIVIDAD"].isin(['ESTUDIANTE'])) & ~(visas["ESTUDIOS"].isin(['PREBASICO','BASICO', 'MEDIO', 'NO INFORMA', 'NINGUNO']))].head(10)

## Modificando el Dataframe

No tan solo podemos acceder a los valores, podemos crear nuevas columnas a partir de las actuales, modificar columnas o incluso eliminar columnas que no necesitamos.


Por ejemplo, podemos crear una nueva columna que obtenga la edad de las personas.

In [ ]:
from datetime import datetime

visas["EDAD"] = (datetime.now() - visas["FECH-NAC"]).transform(lambda x: x.days // 365)
visas.head(5)

Podemos modificar columnas, por ejemplo en la lista de _ESTUDIOS_ tiene valores que son "repetidos" pero tienen diferente nombre. Me refiero a __no indica__, __NO INFORMA__. Queremos eliminar estos dos valores y dejar solo uno.

In [ ]:
visas["ESTUDIOS"].replace("no indica", "NO INFORMA", inplace=True)
visas["ESTUDIOS"].value_counts()

También se puede dar el caso que algunas columnas sean innecesarias, por ejemplo el __AÑO__, ya que en este caso siempre es 2018.

In [ ]:
visas.drop(columns=["AÑO", "FECH-NAC", "TIT_DEP"], inplace=True)
visas.head(5)

## Agrupando

Finalmente podemos agrupar los valores por datos, por ejemplo queremos saber como se distribuyen los inmigrantes por región.

In [ ]:
regiones = visas.groupby("REGION")
type(regiones)

In [ ]:
regiones.size()

In [ ]:
print(type(regiones.get_group("REGION DE ATACAMA")))
regiones.get_group("REGION DE ATACAMA").head(5)

In [ ]:
for region, df in regiones:
    display(HTML(f"<h3 align='center'>{region}</h3>"))
    display(pd.crosstab(df["PAIS"],df["COMUNA"]))

<h3>Visualizaciones</h3>

Si quieres mostrar resultados, lo mejor es mostrarlo a través de gráficos por tres razones:
* Resumir la información
* Se pueden encontrar patrones
* Es más fácil de entender cuando compartas tus resultados

#### Inmigrantes por país

Crearemos un gráfico de pastel (pie, torta, circular, ...) para visualizar de dónde viene la mayor parte de los inmigrantes.

Para esto usaremos una nueva herramienta de visualización... **plotly**:

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode()

**plotly** es algo más difícil de usar que **matplotlib**, sin embargo es mucho más poderoso en cuanto a interactividad. Hay que tener en cuenta lo siguiente:

1. Para funcionar en Jupyter Notebook, ha de llamarse primero a la función `init_notebook_mode()`.
2. Los gráficos son llamados **graph objects**.
2. La función `iplot` permite visualizar gráficos y reciben un diccionario con la lista de **graph objects** a mostrar.

Primero, tendremos una serie con los nombres de los países y otra con la cantidad de inmigrantes pertenecientes a cada país:

In [ ]:
paises = df["PAIS"].value_counts()
paises.values

In [ ]:
THRESHOLD = .01 # porcentaje minimo para aparecer en el pie
mask = (paises.values / paises.values.sum()) > THRESHOLD

labels = list(paises.index[mask]) + ["Otros"]
values = list(paises.values[mask]) + [paises.values[~mask].sum()]

Luego entregaremos estos valores a un **graph object** de un gráfico de pastel:

In [ ]:
#pieses = go.Pie(labels=paises.index, values=paises.values)
pieses = go.Pie(labels=labels, values=values)

Finalmente, la visualización:

In [ ]:
iplot(dict(data=[pieses]))

### Inmigrantes por región

In [ ]:
REGION_ACTUAL = "REGION DE ATACAMA"
region = regiones.get_group(REGION_ACTUAL)

In [ ]:
paises = region.groupby("PAIS")

data = list()
for pais, df in paises:
    count = df["COMUNA"].value_counts()
    trace = go.Bar(
        x = count.index,
        y = count.values,
        name = pais
    )
    data.append(trace)

In [ ]:
layout = go.Layout(
    barmode='stack',
    title = REGION_ACTUAL
)

In [ ]:
iplot(dict(data=data, layout=layout))

### Piramide de Población

https://plot.ly/python/population-pyramid-charts/

In [ ]:
bins = np.arange(0,100,5)
hombres = pd.cut(visas["EDAD"][visas["SEXO"] == "masculino"], bins)
mujeres = pd.cut(visas["EDAD"][visas["SEXO"] == "femenino"], bins)

h_bins = hombres.value_counts(sort=False) * -1
m_bins = mujeres.value_counts(sort=False)

<div id="caso2"></div>
    
# Caso 2: Yahoo Finances

<img src="https://i0.wp.com/cd1.eju.tv/wp-content/uploads/2016/07/578a40b2c03d5.jpg?resize=830%2C415" width=45%>

En el primer caso utilizaremos datos de [__Yahoo Finances__](https://finance.yahoo.com/). En concreto, analizaremos las acciones de [_Nintendo_](https://www.nintendo.co.jp/) entre los años 2013 a 2018.

_Puedes descargar este dataset desde este [link](https://finance.yahoo.com/quote/7974.T/history?period1=1385607600&period2=1543374000&interval=1d&filter=history&frequency=1d)._


In [ ]:
ntdo_file = "./data/NINTENDO.csv"

In [ ]:
ntdo = pd.read_csv(ntdo_file, sep=',')

print(f"Cantidad de filas: {len(ntdo)}")
print(f"Dimensiones: {ntdo.shape}")
print(f"Cantidad de valores: {ntdo.size}")
print()

ntdo.info()
ntdo.head(5)

In [ ]:
ntdo["Date"].dtype

In [ ]:
ntdo["Date"] = pd.to_datetime(ntdo["Date"], format='%Y-%m-%d')
ntdo["Date"].dtype

In [ ]:
ntdo.set_index("Date", inplace=True)
ntdo.head(5)

O al momento de cargar, podemos usar:

In [ ]:
ntdo = pd.read_csv(ntdo_file, index_col="Date", parse_dates=['Date'], date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d'))

ntdo.info()
display(ntdo.head(5))

In [ ]:
ntdo.describe()

In [ ]:
ntdo[["Open", "High", "Close"]].plot.line(figsize=(15,5))
plt.title("Nintendo");

In [ ]:
daterange = ntdo["2016-01-01":"2017-01-01"]
daterange.head(5)

In [ ]:
daterange[["Open", "High", "Close"]].plot.line(figsize=(15,5))
plt.title("Nintendo");

In [ ]:
close = daterange["Close"]

min_value, max_value = close.min(), close.max()
mean = close.mean()
std = close.std()
counts = close.count()
print(f"counts: {counts}\nmin: {min_value}\nmax: {max_value}\nmean: {mean}\nstd: {std}")

In [ ]:
close.plot.line(figsize=(15,5))

plt.plot(close.index, [mean for i in range(counts)],'--b', label=f"Mean: {mean:5.6}")
plt.fill_between(close.index, mean-2*std, mean+2*std, color='b', alpha=0.1)

plt.legend()
plt.title("Nintendo");

In [ ]:
window = 20

daterange = daterange.assign(
                             mean = close.rolling(window).mean(), 
                             std = close.rolling(window).std()
                        )


mean = daterange[["mean"]]
range_std = (daterange["mean"] - 2 * daterange["std"], daterange["mean"] + 2 * daterange["std"])

daterange[["Close"]].plot.line(figsize=(15,10))
plt.plot(daterange[["mean"]],'--b', label="Mean")
plt.fill_between(daterange.index, range_std[0], range_std[1], color='b', alpha=0.1)

plt.legend()
plt.title("Nintendo");

## Volatilidad

In [ ]:
daterange = daterange.assign(volatility = daterange["std"] * np.sqrt(window))

daterange[["volatility"]].plot.line( figsize=(15,5), legend=None, style="--b")
plt.grid()
plt.title("Nintendo")
plt.ylabel("Volatilidad");

## CandleStick Plot

In [ ]:
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates

dat = daterange.reset_index()[["Date", "Open", "High", "Low", "Close"]]
dat["Date"] = dat["Date"].map(mdates.date2num)


f1, ax = plt.subplots(figsize = (15,10))
_ = candlestick_ohlc(ax, dat.values, width=2, colorup='g', alpha =1)

ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
plt.xticks(rotation=45)
#ax.set_xlabel(idx_name)
ax.set_ylabel("OHLC")
ax.set_title("Nintendo");

# Haciendo las visualizaciones más interactivas


## Ideas

* Hacer un plot donde se elija cuales valores mostrar (close, open, high)
* Hacer un plot que se pueda elegir un rango de fechas para mostrar y un subplot donde esté todo (ver imagen)
* Agregarle una opcion para cambiar el tamaño de la ventana del rolling

### IMAGEN V
<img src="https://dz13w8afd47il.cloudfront.net/graphics/9781787123137/graphics/image_15_014.png">

<div id="caso3"></div>

# Caso 3: Imágenes por Resonancia Magnética (MRI para los amigos)

In [ ]:
!ls *.nrrd

Cargamos los datos y su _header_ usando `pynrrd`:

In [ ]:
import nrrd
data, header = nrrd.read("MR-head.nrrd")
data.shape

Tiene 3 dimensiones... **¡Es un cubo de datos! 😧**

<img src="cube.png">

Bueno... podemos visualizar un _slice_ de los datos usando _matplotlib_:

In [ ]:
plt.xticks([]);plt.yticks([])
plt.imshow(data[128,:,:]);

In [ ]:
plt.xticks([]);plt.yticks([])
plt.imshow(data[:,128,:]);

In [ ]:
plt.xticks([]);plt.yticks([])
plt.imshow(data[:,:,65]);

O sumar todos los _slice_ a lo largo de un eje:

In [ ]:
plt.imshow(data.sum(axis=2));

Pero... **¿Cómo ver los datos cómodamente?**

### ¿Qué tal... widgets?

**Widget**: Objeto de Python con el que se puede interactuar y que reacciona a eventos.

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
import IPython.display as ipd # para mostrar los widgets

Creemos un _slider_ que permita elegir un índice en el primer eje (eje x) del cubo de datos.

In [ ]:
slider = widgets.IntSlider(min=0, max=data.shape[0]-1)
slider

¡Se mueve! pero... no hace nada ¬¬

Usaremos...

<div style="text-align:center;font-size:28px">
    <b>¡interact!</b>
</div>

`interact` es una función decorador muy poderoso que utiliza los _widgets_ que se le entreguen para ir cambiando los argumentos de una función dada.

**Warning**: Esto significa que llamará a la función dada cada vez que se cambie el valor de un _widget_.

In [ ]:
@interact(i=slider)
def visualize_x(i):
    plt.imshow(data[i,:,:])

**Warning 2**: Cada slider que viene del mismo objeto está conectado.

In [ ]:
slider

Aunque eso también da ventajas:

In [ ]:
slider.value = 128

Por lo que hay que crear un diferente slider para cada `ìnteract`:

## Isosuperficies

In [ ]:
from plotly import graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

**Curva de Contorno**: Es una curva que pasa por todos los puntos en el espacio que tienen un mismo valor asignado

In [ ]:
s = 180
f = lambda x, y: np.cos((x-s/2)*(y-s/2)/314)
arr = np.fromfunction(f, (s, s))
arr.shape

In [ ]:
plt.imshow(arr)

In [ ]:
lo = go.Layout(width=512, height=512)
cont = go.Contour(z=arr, contours=dict(start=0, end=1, size=0.4, coloring='lines'))
fig = go.Figure([cont], layout=lo)
iplot(fig)

**Isosuperficie (Superficie de Contorno)**: Es una superficie que pasa por todos los puntos en el espacio por el cual su valor tiene el mismo valor.

In [ ]:
from skimage import measure, transform

Escalemos la imagen para que sea más fácil de procesar:

In [ ]:
cdata = transform.rescale(data, 0.2, mode="reflect")
#cdata = transform.rescale(data, 0.2, anti_aliasing=True, anti_aliasing_sigma=1.1, multichannel=False, mode="reflect")
cdata.shape

In [ ]:
plt.imshow(cdata[:,25,:])

**Marching Cubes:** Algoritmo para obtener isosuperficies.

In [ ]:
verts, simplices = measure.marching_cubes_classic(cdata, cdata.mean())

_plotly_ tiene una herramienta para crear una visualización 3D de la superficie a partir de vértices y símplices:

In [ ]:
from plotly import figure_factory as ff
x, y, z = zip(*verts)

#fig = ff.create_trisurf(x=x, y=y, z=z, colormap="Rainbow", simplices=simplices)
fig = ff.create_trisurf(x=x, y=z, z=y, colormap="Rainbow", simplices=simplices)

Por último...

In [ ]:
iplot(fig)

¿Cuáles son nuestros datos?

<div id="summary">
    
# Resumen

<div id="sources"></div>
    
# Recursos en Internet

Para continuar tu camino en convertirte en un verdadero _Data scientist_ te recomendamos las siguentes página y libros que te ayudarán en esta travesía.

### Libros

### Cursos en línea 

### Datasets

### CheatSheets

### Articulos